In [1]:
(ql:quickload "kenzo")
(in-package :cat)

To load "kenzo":
  Load 1 ASDF system:
    kenzo
; Loading "kenzo"



#<PACKAGE "CAT">

# Serre spectral sequence

## Introduction

This chapter is devoted to the `Kenzo` implementation of the Serre spectral sequence with the aim to compute the homology groups of the total space of a fibration. From the programming point of view, this method will be used according to our general scheme of searching homology (generic function `search-efhm`) applied to an object of type `fibration-total`.

## The topological problem

Let ${\cal G}$ be a simplicial group (the fiber space),  $B$, a 1-reduced simplicial set (the base space) and $\tau$ (the fibration), a simplicial morphism of degree $-1$, and $\tau: B \rightarrow \mathcal{G}$ a fibration, that is, a twisting operator as defined in Chapter 12. In the Kenzo program, this object $\tau$ is implemented as an object of class `simplicial-mrph` and "degree" $-1$ while a genuine simplicial morphism has degree 0. We know the program Kenzo knows how to build the total space of the fibration: $B \times_\tau {\cal G}$, which has the same simplices as $B \times {\cal G}$ and the same face operators except the last one, given by:
$$
\partial_n(b,g)  =  (\partial_n b, \tau(b).\partial_n g), \quad b\in B, g \in {\cal G},
$$
where the '`.`' denotes the group operation in ${\cal G}$. This discrepancy between the respective face operators $\partial_n$ induces a discrepancy between the differential operators. We suppose now, that the base and  fiber  spaces are objects of effective homology type, i.e. there exist homotopy equivalences
$$
\begin{aligned}
  & \qquad\widehat{B}  & & & & \qquad\widehat{\cal G} & & \cr
 \rho_1 \swarrow\nearrow  & & \searrow\nwarrow \rho_2 &\quad & \rho'_1\swarrow\nearrow & & \searrow\nwarrow\rho'_2 & \cr
B  & & EB & & {\cal G}   & & EG \cr
\end{aligned}
$$
where the chain complexes $EB$ and $EG$ are effective. The problem consists in finding a homotopy equivalence between $B \times_\tau {\cal G}$ and an effective chain complex noted $EB \widetilde{\otimes} EG$; the underlying graded module of $EB \widetilde{\otimes} EG$ is $EB \otimes EG$ but the differential is twisted in a rather complicated way. In fact, the chain complex $EB \widetilde{\otimes} EG$ is nothing but the Hirsch complex (**Guy Hirsch.** `Sur les groupes d'homologie des espaces fibres`, Bulletin de la Societe Mathematique de Belgique, 1954, vol. 6, pp. 79-96.).

### The Ronald Brown reduction

We start from the Eilenberg-Zilber reduction  (built in `Kenzo` by the function `ez`).
$$
\begin{aligned}
{{\cal C}_*(B \times {\cal G})} & \qquad\stackrel{h}{\longrightarrow} & {}^s{{\cal C}_*(B \times {\cal G})} \cr
 {\scriptstyle f} \downarrow \uparrow {\scriptstyle g}  & & \cr
 {{\cal C}_*(B) \otimes {\cal C}_*({\cal G})} & &\cr
\end{aligned}
$$
Using the *basic perturbation lemma*, we then obtain a new reduction between twisted objects:
$$
\begin{aligned}
{{\cal C}_*(B \times_\tau {\cal G})} & \qquad\stackrel{h}{\longrightarrow} & {}^s{{\cal C}_*(B \times_\tau {\cal G})} \cr
 {\scriptstyle f} \downarrow \uparrow {\scriptstyle g} & &  \cr
 {{\cal C}_*(B) \otimes_t {\cal C}_*({\cal G})} & &\cr
\end{aligned}
$$
where the symbol $\otimes_t$ represents a *twisted tensor product*, induced by the twisting operator $\tau$; this twisting operator is nothing but the Shih twisting cochain (**Gugenheim.**, *On the chain complex of a fibration*. Illinois Journal of Mathematics, 1972, vol. 16, pp. 398-414.) determined by Szczarba (**R. H. Szczarba.** *The homology of twisted cartesian products*. Transactions of the American Mathematical Society, 1961, vol. 100, pp. 197-216.). In effect, writing 
$$ d_\tau = d + (d_\tau -d),$$
one may consider that the differential $d_\tau$ of $B \times_\tau {\cal G}$ is the differential $d$ of $B \times {\cal G}$ modified by the perturbation $d_\tau - d$. The perturbation morphism $d_\tau -d$ is deduced from the  difference  between both  $\partial_n$ face operators. This being done,  a perturbation may possibly be propagated along the Eilenberg-Zilber reduction  (see the method `add` applied to a reduction) and  the perturbation lemma  gives also, as a bonus, the perturbation to be applied to the differential of $B \otimes {\cal G}$  to obtain the differential of $B \otimes_t {\cal G}$.

### The Gugenheim algorithm

On the other hand, from the homotopy equivalences
$$
\begin{aligned}
  & \qquad\widehat{B} & & & & \qquad\widehat{\cal G} & & \cr
 \rho_1 \swarrow\nearrow  & & \searrow\nwarrow \rho_2 &\quad & \rho'_1\swarrow\nearrow & & \searrow\nwarrow\rho'_2 & \cr
B  & & EB & & {\cal G}   & & EG \cr
\end{aligned}
$$
it is possible to build their tensor product, i.e. the homotopy equivalence:
$$
\begin{aligned}
  & \qquad\widehat{B}\otimes \widehat{\cal G} &    \cr
 \swarrow\nearrow & & \searrow\nwarrow \qquad\cr
B\otimes {\cal G}  & & EB \otimes EG \cr
\end{aligned}
$$
This is done in `Kenzo` by the method  `tnsr-prdc`.
In fact, we are interested in the following one:
$$
\begin{aligned}
  & \qquad\widehat{B}\widetilde{\otimes}\widehat{\cal G} &    \cr
 \swarrow\nearrow & & \searrow\nwarrow \qquad\cr
B\otimes_t {\cal G}  & & EB \widetilde{\otimes} EG \cr
\end{aligned}
$$
where $\widehat{B}\widetilde{\otimes}\widehat{\cal G}$ and $EB \widetilde{\otimes} EG$ are appropriate twisted tensor products determined by the perturbation lemma: this last homotopy equivalence is obtained by propagating over the previous one, the perturbation to be applied to the differential of $B \otimes {\cal G}$  to obtain the differential of $B \otimes_t {\cal G}$. The homotopy equivalence so obtained is called in `Kenzo` the *right Serre homotopy equivalence*.

### Assembling the puzzle

Let us build now the so-called *left Serre homotopy equivalence*, 
$$
\begin{aligned}
  & \qquad B\times_\tau{\cal G} &    \cr
 \swarrow\nearrow & & \searrow\nwarrow \qquad\cr
B\times_\tau {\cal G}  & & B\otimes_t {\cal G} \cr
\end{aligned}
$$
where the left reduction is the trivial one and the right reduction is the Brown reduction. We may then compose both Serre homotopy equivalences to obtain what we were looking for: 
$$
\begin{aligned}
  & \qquad B\times_\tau {\cal G}  & & &\widehat{B}\otimes_t\widehat{\cal G} \qquad& & \cr
 \rho_1 \swarrow\nearrow  & & \searrow\nwarrow \rho_2\qquad &\qquad\rho'_1\swarrow\nearrow & & \searrow\nwarrow\rho'_2 & \cr
B\times_\tau {\cal G}  & &B\otimes_t {\cal G}\quad &\quad B\otimes_t {\cal G} & &EB \otimes_t EG \cr
\end{aligned}
$$

## The functions for the Serre spectral sequence

The following functions are given for information purpose.

`fibration-dtau-d-intr` *fibration* *[Function]*
> Build the lisp function corresponding to the difference $d_\tau - d$, where $d$ is the differential in $B \times {\cal G}$ and $d_\tau$ the differential in $B \times_\tau {\cal G}$. This is an internal function and is used in the following one.

`fibration-dtau-d` *fibration* *[Function]*
> Build the morphism $d_\tau-d$, which is the perturbation morphism to be added to the differential  $d$ of $B \times {\cal G}$  to obtain  $d_\tau$, the differential in $B \times_\tau {\cal G}$. We recall that both simplicial sets have the same simplices. The source and the target of this morphism is the total space $B \times_\tau {\cal G}$ and the degree is $-1$.

`Brown-reduction` *fibration* *[Function]*
> Return two values: a) the Brown reduction and b) the  perturbation to be applied to the differential of the tensor product $B \otimes {\cal G}$  to obtain the differential of the twisted tensor product $B \otimes_t {\cal G}$.

`left-Serre-efhm` *fibration* *[Function]*
> Build the left Serre homotopy equivalence from the argument *fibration*.

`right-Serre-efhm` *fibration* *[Function]*
> Build the right Serre homotopy equivalence from the argument *fibration*.

## Searching homology for fibration total spaces

The comment list of a `Kenzo` object which is the total space of a fibration has the form `(fibration-total` *fibration*`)`. The `search-efhm` method applied to a simplicial set having this kind of comment list, builds the composition of the left and right Serre homotopy equivalences. The right one needs the homotopy equivalences attached respectively to the base and  fiber spaces. This may involve a recursive call of `search-efhm`. If the base space or the fiber space are locally effective, the total space is locally effective and the method cannot compute the homology.
```commonlisp
(defmethod SEARCH-EFHM (smst (orgn (eql 'fibration-total)))
  (declare
     (type simplicial-set smst))
  (the homotopy-equivalence
     (fibration-total-efhm (second (orgn smst)))))

(defun FIBRATION-TOTAL-EFHM (fibration)
  (declare (type fibration fibration))
  (the homotopy-equivalence
       (cmps (left-serre-efhm fibration)
             (right-serre-efhm fibration))))
```

### Examples

We begin by some examples similar to examples that we have seen in the fibrations chapter. For the user, the delicate point is to write in Lisp a correct twisting operator. There is no check in this version of `Kenzo` for the coherency of the twisting operator. In the first example we define a fibration $\tau: S^2 \rightarrow K(\mathbb{Z},1)$ with $\tau(s2)= (2)$, then we build the total space of the fibration, namely $P^3\mathbb{R}$, and we compute some known homology groups.

In [2]:
(def sph2 (sphere 2))

[K1 Simplicial-Set]

In [3]:
(def kz1 (k-z-1)) 

[K6 Abelian-Simplicial-Group]

In [4]:
(def tw2 (build-smmr
          :sorc sph2
          :trgt kz1
          :degr -1
          :sintr #'(lambda (dmns gmsm)
                           (declare (ignore gmsm))
                        (unless (= dmns 2)
                                (error "Dimension error for 
                                        the twisting operator S2-->KZ1"))
                        (absm 0 (list 2)))
          :orgn '(s2-tw-kz1)))

[K18 Fibration K1 -> K6]

In [5]:
(? tw2 2 's2)

<AbSm - (2)>

In [6]:
(? tw2 0 '*)

SIMPLE-ERROR: 
Dimension error for 
                                        the twisting operator S2-->KZ1


NIL

In [7]:
(def p3r (fibration-total tw2))

[K24 Simplicial-Set]

In [8]:
(homology p3r 0 10)


Computing boundary-matrix in dimension 0.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 0) :
<TnPr * *> 
End of computing.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 1) :
<TnPr * S1> 
End of computing.




Homology in dimension 0 :


Component Z


---done---

;; Clock -> 2017-10-08, 12h 30m 3s.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 1) :
<TnPr * S1> 
End of computing.


Computing boundary-matrix in dimension 2.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 2) :
<TnPr S2 *> 
End of computing.




Homology in dimension 1 :


Component Z/2Z


---done---

;; Clock -> 2017-10-08, 12h 30m

NIL

Let us do the same kind of computations with $\tau(s2)= (5)$ and $\tau(s2)= (17)$.

In [9]:
(def tw5 (build-smmr
             :sorc sph2
             :trgt kz1
             :degr -1
             :sintr #'(lambda (dmns gmsm)
                              (declare (ignore dmns gmsm))
                        (absm 0 (list 5)))
             :orgn '(s2-tw-kz1-5))) 

[K110 Fibration K1 -> K6]

In [10]:
(def total-5 (fibration-total tw5))

[K111 Simplicial-Set]

In [11]:
(homology total-5 0 5)


Computing boundary-matrix in dimension 0.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 0) :
<TnPr * *> 
End of computing.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 1) :
<TnPr * S1> 
End of computing.




Homology in dimension 0 :


Component Z


---done---

;; Clock -> 2017-10-08, 12h 30m 3s.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 1) :
<TnPr * S1> 
End of computing.


Computing boundary-matrix in dimension 2.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 2) :
<TnPr S2 *> 
End of computing.




Homology in dimension 1 :


Component Z/5Z


---done---

;; Clock -> 2017-10-08, 12h 30m

NIL

In [12]:
(def tw17 (build-smmr
             :sorc sph2
             :trgt kz1
             :degr -1
             :sintr #'(lambda (dmns gmsm)
                              (declare (ignore dmns gmsm))
                        (absm 0 (list 17)))
             :orgn '(s2-tw-kz1-17)))

[K165 Fibration K1 -> K6]

In [13]:
(def total-17 (fibration-total tw17)) 

[K166 Simplicial-Set]

In [14]:
(homology total-17 0 4)


Computing boundary-matrix in dimension 0.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 0) :
<TnPr * *> 
End of computing.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 1) :
<TnPr * S1> 
End of computing.




Homology in dimension 0 :


Component Z


---done---

;; Clock -> 2017-10-08, 12h 30m 3s.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 1) :
<TnPr * S1> 
End of computing.


Computing boundary-matrix in dimension 2.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 2) :
<TnPr S2 *> 
End of computing.




Homology in dimension 1 :


Component Z/17Z


---done---

;; Clock -> 2017-10-08, 12h 30

NIL

Using the last total space (`total-17`), it is instructive to enter in the details of the resulting homotopy equivalence that the system builds to compute the homology groups. First, the function `echcm` selects the **effective** chain complex of the homotopy equivalence, value of the slot `efhm` of the simplicial set `total-17`, and we may print some basis of this chain complex. In fact, all basis in dimension above 3 are null. The presence of the symbol `S1` is due to the fact that the effective chain complex used for the homology of $K(\mathbb{Z},1)$ is the circle (see the classifying spaces chapter).

In [15]:
(def echcm (echcm total-17))

[K205 Chain-Complex]

In [16]:
(dotimes (i 6) (print (basis echcm i)))


(<TnPr * *>) 
(<TnPr * S1>) 
(<TnPr S2 *>) 
(<TnPr S2 S1>) 
NIL 
NIL 

NIL

The resulting homotopy equivalence built by the system, may be summarized by the diagram:
$$
\begin{aligned}
  &\qquad\qquad \widehat{S^2\times_\tau K(\mathbb{Z},1)} &    \cr
 f\swarrow\nearrow & & \searrow\nwarrow g \qquad\qquad \cr
S^2\times_\tau K(\mathbb{Z},1) & & [K205\, Chain\, Complex] \cr
\end{aligned}
$$
where *f* and *g* are respectively the *descending* and *ascending* morphisms of the left and right reductions of the homotopy equivalence. We recall that these morphisms are obtained respectively by the functions `lf` and `rg`. We may
compose these morphisms to create a morphism `fg` from $[K205]$ to $S^2\times_\tau K(\mathbb{Z},1)$ and see how it acts on the basis  elements in dimension 1 and 2:

In [17]:
(def fg (cmps (lf (efhm total-17)) (rg (efhm total-17))))

[K220 Morphism (degree 0): K205 -> K166]

In [18]:
(def gen1 (first (basis echcm 1)))

<TnPr * S1>

In [19]:
(? fg 1 gen1)


----------------------------------------------------------------------{CMBN 1}
<-1 * <CrPr 0 * - (1)>>
------------------------------------------------------------------------------


In [20]:
(def gen2 (first (basis echcm 2)))

<TnPr S2 *>

In [21]:
(def fg-gen2 (? fg 2 gen2))


----------------------------------------------------------------------{CMBN 2}
<-1 * <CrPr - S2 1-0 NIL>>
<-1 * <CrPr 1-0 * - (1 1)>>
<-1 * <CrPr 1-0 * - (1 2)>>
<-1 * <CrPr 1-0 * - (1 3)>>
<-1 * <CrPr 1-0 * - (1 4)>>
<-1 * <CrPr 1-0 * - (1 5)>>
<-1 * <CrPr 1-0 * - (1 6)>>
<-1 * <CrPr 1-0 * - (1 7)>>
<-1 * <CrPr 1-0 * - (1 8)>>
<-1 * <CrPr 1-0 * - (1 9)>>
... ...
------------------------------------------------------------------------------


The boundary of this combination in $S^2\times_\tau K(\mathbb{Z},1)$ is

In [22]:
(? total-17 *)


----------------------------------------------------------------------{CMBN 1}
<-17 * <CrPr 0 * - (1)>>
------------------------------------------------------------------------------


to which corresponds in the effective chain complex, the boundary of the basis element in dimension 2, $gen2=s2 \otimes *$:

In [23]:
(? echcm 2 gen2)


----------------------------------------------------------------------{CMBN 1}
<17 * <TnPr * S1>>
------------------------------------------------------------------------------


The printing of the 3 faces of the 17 simplices of the combination `fg-gen2` shows the geometrical organisation of the twisted product:

In [24]:
(dolist(iterm (cmbn-list fg-gen2))
       (dotimes (i 3)(print (face total-17 i 2 (gnrt iterm)))) 
                     (terpri))


<AbSm 0 <CrPr - * - NIL>> 
<AbSm 0 <CrPr - * - NIL>> 
<AbSm - <CrPr 0 * - (17)>> 

<AbSm - <CrPr 0 * - (1)>> 
<AbSm - <CrPr 0 * - (2)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (2)>> 
<AbSm - <CrPr 0 * - (3)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (3)>> 
<AbSm - <CrPr 0 * - (4)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (4)>> 
<AbSm - <CrPr 0 * - (5)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (5)>> 
<AbSm - <CrPr 0 * - (6)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (6)>> 
<AbSm - <CrPr 0 * - (7)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (7)>> 
<AbSm - <CrPr 0 * - (8)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (8)>> 
<AbSm - <CrPr 0 * - (9)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (9)>> 
<AbSm - <CrPr 0 * - (10)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (10)>> 
<AbSm - <CrPr 0 * - (11)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <CrPr 0 * - (11)>> 
<AbSm - <CrPr 0 * - (12)>> 
<AbSm - <CrPr 0 * - (1)>> 

<AbSm - <

NIL

We have seen that for the special case of loop spaces, there is a canonical fibration and that the function `twisted-crts-prdc` builds the total space $X \times_\tau \Omega X$.

In [25]:
(def sph3 (sphere 3))

[K221 Simplicial-Set]

In [26]:
(def tw3 (twisted-crts-prdc sph3))

[K243 Simplicial-Set]

In [27]:
(face tw3 3 3 (crpr 0 's3 7 +null-loop+))

<AbSm - <CrPr 1-0 * - <<Loop[S3]>>>>

It is of course possible to study, any other fibration, provided one defines a correct fibration $\tau$. In the following example the fibration $\tau: S^3 \longrightarrow \Omega S^3$, is defined by $\tau(s3)= s3^{-2},$ where $s3^{-2}$ is a word in the Kan simplicial version of the first loop space of $S^3$. In Kenzo, this object is created by the statement:

In [28]:
(absm 0 (loop3 0 's3 -2))

<AbSm - <<Loop[S3\-2]>>>

In [29]:
(def tws3 (build-smmr   :sorc sph3
                         :trgt (loop-space sph3)
                         :degr -1
                         :sintr #'(lambda (dmns gmsm) (declare (ignore dmns gmsm)) (absm 0 (loop3 0 's3 -2)))
                         :orgn '(s3-tw-lps3)))

[K248 Fibration K221 -> K226]

In [30]:
(def total (fibration-total tws3))

[K249 Simplicial-Set]

In [31]:
(homology total 0 7)


Computing boundary-matrix in dimension 0.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 0) :
<TnPr * <<AlLp>>> 
End of computing.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 0.




Homology in dimension 0 :


Component Z


---done---

;; Clock -> 2017-10-08, 12h 30m 3s.


Computing boundary-matrix in dimension 1.
Rank of the source-module : 0.


Computing boundary-matrix in dimension 2.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 2) :
<TnPr * <<AlLp[2 S3]>>> 
End of computing.




Homology in dimension 1 :



---done---

;; Clock -> 2017-10-08, 12h 30m 3s.


Computing boundary-matrix in dimension 2.
Rank of the source-module : 1.


;; Clock -> 2017-10-08, 12h 30m 3s.
Computing the boundary of the generator 1/1 (dimension 2) :
<TnPr * <<AlLp[2 S3]>>> 
End of computing.


Computing boundary-matrix in dim

NIL

### Lisp files concerned in this chapter

`serre.lisp`, `searching-homology.lisp`.